# Decoding with ANOVA + SVM: Chocolate Data -- imagine data (full session)
### This is a simple decoding using feature selection followed by an SVM.  

Reference: http://nilearn.github.io/auto_examples/02_decoding/plot_haxby_anova_svm.html#sphx-glr-auto-examples-02-decoding-plot-haxby-anova-svm-py

In [ ]:
#this imports all the commands needed for the script to work#
import os
import numpy as np
import nilearn
import glob
#import matplotlib
import nibabel as nib
import pandas as pd 
from nilearn.input_data import NiftiMasker 

In [ ]:
average_ana=os.path.join('/projects/niblab/nilearn_projects/CS_avg_mprage_image.nii.gz')
#image mask
imag_mask=os.path.join('/projects/niblab/nilearn_projects/power_roimask_4bi.nii.gz')
#our behavioral csv file 
stim = os.path.join('/projects','niblab','scripts','nilean_stuff','label_all_sub.csv')
#our dataset concatenated image 
dataset='/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/group_ana/w1_imagine_all.nii.gz'

#load behavioral data into a pandas df
behavioral = pd.read_csv(stim, sep=",")
#grab conditional labels 
conditions = behavioral["label"]

#restrict data to our target analysis - app vs h2O
condition_mask = behavioral["label"].isin(["app", "H2O"])
conditions = conditions[condition_mask]

#confirm we have the # of condtions needed
print(conditions.unique())

['H2O' 'app']

In [ ]:
# Record these as an array of sessions, with fields for condition and run 
session = behavioral[condition_mask].to_records(index=False)
print(session.dtype.names)

('labels', 'subs')

In [ ]:
# for decoding, standardizing the data is often very important  
# if data hasn't been smoothed, you can do that here too  
masker = NiftiMasker(mask_img=imag_mask, standardize=True, memory="nilearn_cache", memory_level=1)
X = masker.fit_transform(dataset)
#Apply our condition mask
X=X[condition_mask]

### Build the Decoder

In [ ]:
# Define the prediction function to be used. Here we use a SVC with a linear kernel 
# Here we are using a 'rbf' kernel, and we set the decision function shape to 'ovr', standing for 'one vs rest'
from sklearn.svm import SVC
svc = SVC(kernel='rbf', decision_function_shape='ovr')

# Now we define the dimension reduction to be used. 
# Here we use a classical univariate feature selection based on F-test, namely Anova.
# When doing full brain analysis, it's best to use SelectPercentile, keeping 5% of the voxels 
# (because it is independent of the resolution of the data )
from sklearn.feature_selection import SelectPercentile, f_classif 
feature_selection = SelectPercentile(f_classif, percentile=5)


# We now plug our classifier and our feature selection into a pipeline that performs 2 operations successively:
from sklearn.pipeline import Pipeline
anova_svc = Pipeline([('anova', feature_selection), ('svc', svc)])

### Fit the decoder and predict

In [ ]:
anova_svc.fit(X, conditions)
y_pred = anova_svc.predict(X)

### Obtain the prediction scores through Cross Validation 

In [ ]:
from sklearn.model_selection import LeaveOneGroupOut, cross_val_score

# Define the cross validation scheme used for validation. 
# Here we use the LeaveOneGroupOut CV on the session group, which corresponds to a Leave-One-session-out
cv = LeaveOneGroupOut()

# Compute the prediction accuracy for the different folds(i.e session)
cv_scores = cross_val_score(anova_svc, X, conditions, cv=cv, groups=session)

#Return the corresponding mean prediction accuracy
classification_accuracy = cv_scores.mean()

# print the results:
print("Classification accuracy: %.4f / Chance level: %f" %
      (classification_accuracy, 1. / len(conditions.unique())))


Classification accuracy: 0.0029 / Chance level: 0.500000

In [ ]:
coef = svc.coef_
# reverse feature selection
coef = feature_selection.inverse_transform(coef)
# reverse masking
weight_img = masker.inverse_transform(coef)


# Use the mean image as a background to avoid relying on anatomical data
from nilearn import image
mean_img = image.mean_img(func_filename)

# Create the figure
from nilearn.plotting import plot_stat_map, show
plot_stat_map(weight_img, mean_img, title='SVM weights')

# Saving the results as a Nifti file may also be important
weight_img.to_filename('haxby_face_vs_house.nii')


show()

In [ ]:
#load behavioral data into a pandas df
behavioral = pd.read_csv(stim, sep=",")
#grab conditional labels 
conditions = behavioral["label"]

#restrict data to our target analysis - app vs h2O
condition_mask = behavioral["label"].isin(["unapp", "app", "H2O"])
conditions = conditions[condition_mask]

#confirm we have the # of condtions needed
print(conditions.unique())
# Record these as an array of sessions, with fields for condition and run 
session = behavioral[condition_mask].to_records(index=False)
print(session.dtype.names)
# for decoding, standardizing the data is often very important  
# if data hasn't been smoothed, you can do that here too  
masker = NiftiMasker(mask_img=imag_mask, standardize=True, memory="nilearn_cache", memory_level=1)
X = masker.fit_transform(dataset)
#Apply our condition mask
X=X[condition_mask]
# Define the prediction function to be used. Here we use a SVC with a linear kernel 
# Here we are using a 'rbf' kernel, and we set the decision function shape to 'ovr', standing for 'one vs rest'
from sklearn.svm import SVC
svc = SVC(kernel='rbf', decision_function_shape='ovr')

# Now we define the dimension reduction to be used. 
# Here we use a classical univariate feature selection based on F-test, namely Anova.
# When doing full brain analysis, it's best to use SelectPercentile, keeping 5% of the voxels 
# (because it is independent of the resolution of the data )
from sklearn.feature_selection import SelectPercentile, f_classif 
feature_selection = SelectPercentile(f_classif, percentile=5)


# We now plug our classifier and our feature selection into a pipeline that performs 2 operations successively:
from sklearn.pipeline import Pipeline
anova_svc = Pipeline([('anova', feature_selection), ('svc', svc)])

anova_svc.fit(X, conditions)
y_pred = anova_svc.predict(X)

from sklearn.model_selection import LeaveOneGroupOut, cross_val_score

# Define the cross validation scheme used for validation. 
# Here we use the LeaveOneGroupOut CV on the session group, which corresponds to a Leave-One-session-out
cv = LeaveOneGroupOut()

# Compute the prediction accuracy for the different folds(i.e session)
cv_scores = cross_val_score(anova_svc, X, conditions, cv=cv, groups=session)

#Return the corresponding mean prediction accuracy
classification_accuracy = cv_scores.mean()

# print the results:
print("Classification accuracy: %.4f / Chance level: %f" %
      (classification_accuracy, 1. / len(conditions.unique())))


Classification accuracy: 0.0002 / Chance level: 0.333333

In [ ]:
#load behavioral data into a pandas df
behavioral = pd.read_csv(stim, sep=",")
#grab conditional labels 
conditions = behavioral["label"]

#restrict data to our target analysis - app vs h2O
condition_mask = behavioral["label"].isin(["unapp", "app", "H2O"])
conditions = conditions[condition_mask]

#confirm we have the # of condtions needed
print(conditions.unique())
# Record these as an array of sessions, with fields for condition and run 
session = behavioral[condition_mask].to_records(index=False)
print(session.dtype.names)
# for decoding, standardizing the data is often very important  
# if data hasn't been smoothed, you can do that here too  
masker = NiftiMasker(mask_img=imag_mask, standardize=True, memory="nilearn_cache", memory_level=1)
X = masker.fit_transform(dataset)
#Apply our condition mask
X=X[condition_mask]
# Define the prediction function to be used. Here we use a SVC with a linear kernel 
# Here we are using a 'rbf' kernel, and we set the decision function shape to 'ovr', standing for 'one vs rest'
from sklearn.svm import SVC
svc = SVC(kernel='rbf')

# Now we define the dimension reduction to be used. 
# Here we use a classical univariate feature selection based on F-test, namely Anova.
# When doing full brain analysis, it's best to use SelectPercentile, keeping 5% of the voxels 
# (because it is independent of the resolution of the data )
from sklearn.feature_selection import SelectPercentile, f_classif 
feature_selection = SelectPercentile(f_classif, percentile=5)


# We now plug our classifier and our feature selection into a pipeline that performs 2 operations successively:
from sklearn.pipeline import Pipeline
anova_svc = Pipeline([('anova', feature_selection), ('svc', svc)])

anova_svc.fit(X, conditions)
y_pred = anova_svc.predict(X)

from sklearn.model_selection import LeaveOneGroupOut, cross_val_score

# Define the cross validation scheme used for validation. 
# Here we use the LeaveOneGroupOut CV on the session group, which corresponds to a Leave-One-session-out
cv = LeaveOneGroupOut()

# Compute the prediction accuracy for the different folds(i.e session)
cv_scores = cross_val_score(anova_svc, X, conditions, cv=cv, groups=session)

#Return the corresponding mean prediction accuracy
classification_accuracy = cv_scores.mean()

# print the results:
print("Classification accuracy: %.4f / Chance level: %f" %
      (classification_accuracy, 1. / len(conditions.unique())))


Classification accuracy: 0.0002 / Chance level: 0.333333

In [ ]:
#load behavioral data into a pandas df
behavioral = pd.read_csv(stim, sep=",")
#grab conditional labels 
conditions = behavioral["label"]

#restrict data to our target analysis - app vs h2O
condition_mask = behavioral["label"].isin("app", "H2O"])
conditions = conditions[condition_mask]

#confirm we have the # of condtions needed
print(conditions.unique())
# Record these as an array of sessions, with fields for condition and run 
session = behavioral[condition_mask].to_records(index=False)
print(session.dtype.names)
# for decoding, standardizing the data is often very important  
# if data hasn't been smoothed, you can do that here too  
masker = NiftiMasker(mask_img=imag_mask, standardize=True, memory="nilearn_cache", memory_level=1)
X = masker.fit_transform(dataset)
#Apply our condition mask
X=X[condition_mask]
# Define the prediction function to be used. Here we use a SVC with a linear kernel 
# Here we are using a 'rbf' kernel, and we set the decision function shape to 'ovr', standing for 'one vs rest'
from sklearn.svm import SVC
svc = SVC(kernel='linear')

# Now we define the dimension reduction to be used. 
# Here we use a classical univariate feature selection based on F-test, namely Anova.
# When doing full brain analysis, it's best to use SelectPercentile, keeping 5% of the voxels 
# (because it is independent of the resolution of the data )
from sklearn.feature_selection import SelectPercentile, f_classif 
feature_selection = SelectPercentile(f_classif, percentile=5)


# We now plug our classifier and our feature selection into a pipeline that performs 2 operations successively:
from sklearn.pipeline import Pipeline
anova_svc = Pipeline([('anova', feature_selection), ('svc', svc)])

anova_svc.fit(X, conditions)
y_pred = anova_svc.predict(X)

from sklearn.model_selection import LeaveOneGroupOut, cross_val_score

# Define the cross validation scheme used for validation. 
# Here we use the LeaveOneGroupOut CV on the session group, which corresponds to a Leave-One-session-out
cv = LeaveOneGroupOut()

# Compute the prediction accuracy for the different folds(i.e session)
cv_scores = cross_val_score(anova_svc, X, conditions, cv=cv, groups=session)

#Return the corresponding mean prediction accuracy
classification_accuracy = cv_scores.mean()

# print the results:
print("Classification accuracy: %.4f / Chance level: %f" %
      (classification_accuracy, 1. / len(conditions.unique())))


Classification accuracy: 0.1089 / Chance level: 0.500000

As we can see, using SVM with Anova and a selectkpercentile we are getting terrible accuracy results. We tried with both a RBF and linear kernel.  
  
    
This next process we increase the percentile to 15%

In [ ]:
#load behavioral data into a pandas df
behavioral = pd.read_csv(stim, sep=",")
#grab conditional labels 
conditions = behavioral["label"]

#restrict data to our target analysis - app vs h2O
condition_mask = behavioral["label"].isin("app", "H2O"])
conditions = conditions[condition_mask]

#confirm we have the # of condtions needed
print(conditions.unique())
# Record these as an array of sessions, with fields for condition and run 
session = behavioral[condition_mask].to_records(index=False)
print(session.dtype.names)
# for decoding, standardizing the data is often very important  
# if data hasn't been smoothed, you can do that here too  
masker = NiftiMasker(mask_img=imag_mask, standardize=True, memory="nilearn_cache", memory_level=1)
X = masker.fit_transform(dataset)
#Apply our condition mask
X=X[condition_mask]
# Define the prediction function to be used. Here we use a SVC with a linear kernel 
# Here we are using a 'rbf' kernel, and we set the decision function shape to 'ovr', standing for 'one vs rest'
from sklearn.svm import SVC
svc = SVC(kernel='linear')

# Now we define the dimension reduction to be used. 
# Here we use a classical univariate feature selection based on F-test, namely Anova.
# When doing full brain analysis, it's best to use SelectPercentile, keeping 5% of the voxels 
# (because it is independent of the resolution of the data )
from sklearn.feature_selection import SelectPercentile, f_classif 
feature_selection = SelectPercentile(f_classif, percentile=5)


# We now plug our classifier and our feature selection into a pipeline that performs 2 operations successively:
from sklearn.pipeline import Pipeline
anova_svc = Pipeline([('anova', feature_selection), ('svc', svc)])

anova_svc.fit(X, conditions)
y_pred = anova_svc.predict(X)

from sklearn.model_selection import LeaveOneGroupOut, cross_val_score

# Define the cross validation scheme used for validation. 
# Here we use the LeaveOneGroupOut CV on the session group, which corresponds to a Leave-One-session-out
cv = LeaveOneGroupOut()

# Compute the prediction accuracy for the different folds(i.e session)
cv_scores = cross_val_score(anova_svc, X, conditions, cv=cv, groups=session)

#Return the corresponding mean prediction accuracy
classification_accuracy = cv_scores.mean()

# print the results:
print("Classification accuracy: %.4f / Chance level: %f" %
      (classification_accuracy, 1. / len(conditions.unique())))
